## feature extraction exercise
```This exercise is purely about features extraction. We will learn how to do it quick and efficiently.
We will be working on a kaggle dataset of quora questions, where each record is composed of a pair of questions, while the target is to determine whether the questions have the same meaning.
We will extract features for each question and for each pair of questions and will train a simple model (default xgboost) using those features.```

```The purpose of this exercise is to acquire good practices, so please read the instructions carefully and do as it says. You are also encouraged to look at the solution when after you are finished. In addition, when solving the exercise, try to write
as efficient and as clean code as you can.```

```Note: We are about to do some kaggle cheats, that is, we will compute features by mixing the train and the test.
Please notice exactly where we did so. In addition, every time you meet a question in the instructions (you can identify a question by '?'), please answer it in a comment block. ```

```~Ittai Haran```

In [496]:
# some modules you might find useful

import pandas as pd
import numpy as np
from collections import Counter
from functools import partial
import re
import os

import matplotlib.pyplot as plt
%matplotlib inline

In [497]:
# sentence tokenizer for future use

from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
twt_tokenizer = TweetTokenizer()

# word2vec model for future use

## can be found in: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('D:\Download\GoogleNews-vectors-negative300.bin.gz', binary=True)

In [498]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

In [499]:
data = pd.read_csv('D:/Download/quora-question-pairs/train.csv') 
data.head()
data = data.iloc[:2000]
data = data[data.apply(lambda x: not (type(x['question1']) == float or type(x['question2']) == float), axis = 1)]

In [500]:
data

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
1995,1995,3969,3970,"I am visiting Sri Lanka soonfor 9 days, how can I pick up Sri Lankan girls and have fun?",Do Indians hate Sri Lankans?,0
1996,1996,3971,3972,What are some good examples of 4 stanza poems?,What are some good Ilocano poems?,0
1997,1997,3973,3974,Which CPU is better I3 4th Gen or 6th Gen?,Which is better intel i5 (6th gen) or i7 (5th gen)?,0
1998,1998,3975,3976,What are some of the best tourist places to visit in Chhattisgarh?,Where are the foremost tourist places in Chhattisgarh?,1


```First we would like to extract features regarding a single question. In order to do so, first create a dataset containing  all the questions (and their id. why should we remember the id?), without duplicates. Name it 'questions'.```

In [501]:
q1 = data[['qid1','question1']].rename(columns={'question1' : 'question' ,'qid1':'qID' })
q2 = data[['qid2','question2']].rename(columns={'question2' : 'question' ,'qid2':'qID'})

In [502]:
df_union= pd.concat([q1, q2]).drop_duplicates()
df_union['question'] = df_union['question'].str.lower() #low case

In [503]:
df_union

,qID,question
0,1,what is the step by step guide to invest in share market in india?
1,3,what is the story of kohinoor (koh-i-noor) diamond?
2,5,how can i increase the speed of my internet connection while using a vpn?
3,7,why am i mentally very lonely? how can i solve it?
4,9,"which one dissolve in water quikly sugar, salt, methane and carbon di oxide?"
...,...,...
1995,3970,do indians hate sri lankans?
1996,3972,what are some good ilocano poems?
1997,3974,which is better intel i5 (6th gen) or i7 (5th gen)?
1998,3976,where are the foremost tourist places in chhattisgarh?


``` Add a column containing the questions, tokenized using twt_tokenizer, the TweetTokenizer object we created earlier. Name it 'question_sep'. Make sure that you treat the questions in lower case.```

In [504]:
df_union['question_sep'] = df_union.apply(lambda x: twt_tokenizer.tokenize(x['question']) , axis=1 )

In [505]:
df_union

,qID,question,question_sep
0,1,what is the step by step guide to invest in share market in india?,"[what, is, the, step, by, step, guide, to, invest, in, share, market, in, india, ?]"
1,3,what is the story of kohinoor (koh-i-noor) diamond?,"[what, is, the, story, of, kohinoor, (, koh-i-noor, ), diamond, ?]"
2,5,how can i increase the speed of my internet connection while using a vpn?,"[how, can, i, increase, the, speed, of, my, internet, connection, while, using, a, vpn, ?]"
3,7,why am i mentally very lonely? how can i solve it?,"[why, am, i, mentally, very, lonely, ?, how, can, i, solve, it, ?]"
4,9,"which one dissolve in water quikly sugar, salt, methane and carbon di oxide?","[which, one, dissolve, in, water, quikly, sugar, ,, salt, ,, methane, and, carbon, di, oxide, ?]"
...,...,...,...
1995,3970,do indians hate sri lankans?,"[do, indians, hate, sri, lankans, ?]"
1996,3972,what are some good ilocano poems?,"[what, are, some, good, ilocano, poems, ?]"
1997,3974,which is better intel i5 (6th gen) or i7 (5th gen)?,"[which, is, better, intel, i5, (, 6th, gen, ), or, i7, (, 5th, gen, ), ?]"
1998,3976,where are the foremost tourist places in chhattisgarh?,"[where, are, the, foremost, tourist, places, in, chhattisgarh, ?]"


```Create an empty list called 'question_features_for_future_use'. We are going to befoul the questions dataframe, so we will want to remember which of its columns are important to us and which are just columns helping us to create other columns.```
```Next, I will ask you to create some features. Whenever I use this sign: (*), know that you have to add the feature name to the list.```

In [506]:
question_features_for_future_use = []

```Before we start computing features, write a function that gets a column name and saves a csv file with 2 columns: qid and the column chosen. Name it 'save_feature' and make sure you use it after every feature computed, since it might be very very important for later parts of the exercise and your life.```

```Save the features in the resources/features/<col_name>.csv .```

```use os.path and os.getcwd().```

In [507]:
def save_col_toCSV(col_name):
    df_union.to_csv(index=False , columns=['qID' , col_name] , path_or_buf='resources/features/' + col_name + '.csv' )

```Compute the following:```
- ```Counter of the word part of speech (use collections.Counter and pos_tag, which we imported earlier. do it using one line). (*)```
- ```number of different numbers appearing in the question
(numbers, not digits. use regex. don't count words like 'one') (one line). (*)```
- ```number of words in a question (one line). (*)```
- ```length of longest word (one line). (*)```
- ```word2vec mean of the question. (*)```

In [508]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Melody\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Melody\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [509]:
df_union['part_of_speech_counter'] = df_union.apply(lambda x : Counter([j for i,j in pos_tag(word_tokenize(x['question']))]) , axis=1)
question_features_for_future_use.append('part_of_speech_counter') # (*)

In [510]:
df_union['number_of_numbers'] = df_union.apply(lambda x : len([int(word) for word in x['question'].split(sep=' ') if word.isdigit()]) , axis=1)
question_features_for_future_use.append('number_of_numbers') # (*)

In [511]:
df_union['number_of_words'] = df_union.apply(lambda x: len(x['question_sep']) - 1 , axis=1) #minus 1 beacuse i dont want to count the question mark.
question_features_for_future_use.append('number_of_words') # (*)

In [512]:
df_union['length_of_longest_word'] =df_union['question_sep'].apply(lambda x: max(len(s) for s in x))
question_features_for_future_use.append('length_of_longest_word') # (*)

In [513]:
df_union

,qID,question,question_sep,part_of_speech_counter,number_of_numbers,number_of_words,length_of_longest_word
0,1,what is the step by step guide to invest in share market in india?,"[what, is, the, step, by, step, guide, to, invest, in, share, market, in, india, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 5, 'IN': 3, 'RB': 1, 'TO': 1, 'VB': 1, '.': 1}",0,14,6
1,3,what is the story of kohinoor (koh-i-noor) diamond?,"[what, is, the, story, of, kohinoor, (, koh-i-noor, ), diamond, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 3, 'IN': 1, '(': 1, 'JJ': 1, ')': 1, '.': 1}",0,10,10
2,5,how can i increase the speed of my internet connection while using a vpn?,"[how, can, i, increase, the, speed, of, my, internet, connection, while, using, a, vpn, ?]","{'WRB': 1, 'MD': 1, 'VB': 2, 'DT': 2, 'NN': 3, 'IN': 2, 'PRP$': 1, 'JJ': 1, 'VBG': 1, '.': 1}",0,14,10
3,7,why am i mentally very lonely? how can i solve it?,"[why, am, i, mentally, very, lonely, ?, how, can, i, solve, it, ?]","{'WRB': 2, 'VBP': 1, 'JJ': 1, 'RB': 3, '.': 2, 'MD': 1, 'VB': 2, 'PRP': 1}",0,12,8
4,9,"which one dissolve in water quikly sugar, salt, methane and carbon di oxide?","[which, one, dissolve, in, water, quikly, sugar, ,, salt, ,, methane, and, carbon, di, oxide, ?]","{'WDT': 1, 'CD': 1, 'NN': 8, 'IN': 2, ',': 2, 'CC': 1, '.': 1}",0,15,8
...,...,...,...,...,...,...,...
1995,3970,do indians hate sri lankans?,"[do, indians, hate, sri, lankans, ?]","{'VBP': 1, 'NNS': 3, 'VB': 1, '.': 1}",0,5,7
1996,3972,what are some good ilocano poems?,"[what, are, some, good, ilocano, poems, ?]","{'WDT': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 1, 'NNS': 1, '.': 1}",0,6,7
1997,3974,which is better intel i5 (6th gen) or i7 (5th gen)?,"[which, is, better, intel, i5, (, 6th, gen, ), or, i7, (, 5th, gen, ), ?]","{'WDT': 1, 'VBZ': 1, 'RBR': 1, 'NN': 5, '(': 2, 'CD': 2, ')': 2, 'CC': 1, '.': 1}",0,15,6
1998,3976,where are the foremost tourist places in chhattisgarh?,"[where, are, the, foremost, tourist, places, in, chhattisgarh, ?]","{'WRB': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 2, 'NNS': 1, 'IN': 1, '.': 1}",0,8,12


In [514]:
word2vec.wv[df_union.iloc[0]['question_sep'][8]]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([-5.55419922e-03,  9.61914062e-02,  6.73828125e-02, -4.85839844e-02,
        4.61425781e-02,  2.25585938e-01,  1.06445312e-01, -3.02734375e-01,
        1.48437500e-01,  1.18652344e-01, -1.07421875e-01,  1.87500000e-01,
       -2.94921875e-01,  2.21679688e-01, -1.52343750e-01, -8.30078125e-03,
       -2.59765625e-01,  2.33398438e-01, -1.53320312e-01,  2.51953125e-01,
       -8.00781250e-02,  1.63085938e-01,  1.41601562e-01,  2.40234375e-01,
        7.51953125e-02,  5.37109375e-02,  1.00097656e-02,  5.73730469e-02,
       -1.64062500e-01, -6.64062500e-02,  2.54821777e-03,  6.54296875e-02,
       -3.18359375e-01, -2.13867188e-01,  3.07617188e-02, -1.80664062e-01,
        4.46777344e-02,  1.73339844e-02, -1.53320312e-01,  3.32031250e-01,
        2.40234375e-01,  2.53906250e-01,  8.83789062e-02, -1.36718750e-01,
       -1.34765625e-01, -1.86523438e-01, -1.99218750e-01,  1.97753906e-02,
       -3.20312500e-01,  8.20312500e-02,  3.00781250e-01, -3.75976562e-02,
       -9.91210938e-02, -

In [515]:
def word2vec_mean_of_question(row):
    sum_vector =0
    count =0
    for i in row['question_sep']:
              if i in (word2vec.vocab.keys()):
                sum_vector = sum_vector + word2vec.wv[i]
                count = count + 1

    avg_vec = sum_vector / count
    return word2vec.most_similar(positive=[avg_vec], topn=1)

In [516]:
df_union['word2vec_mean_of_question']=df_union.apply(word2vec_mean_of_question , axis=1).apply(lambda x : [y[0] for y in list(x)])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [517]:
df_union['word2vec_mean_of_question'] = df_union['word2vec_mean_of_question'].str.join(', ')
question_features_for_future_use.append('word2vec_mean_of_question') # (*)

In [518]:
question_features_for_future_use

['part_of_speech_counter',
 'number_of_numbers',
 'number_of_words',
 'length_of_longest_word',
 'word2vec_mean_of_question']

In [519]:
df_union.head()

,qID,question,question_sep,part_of_speech_counter,number_of_numbers,number_of_words,length_of_longest_word,word2vec_mean_of_question
0,1,what is the step by step guide to invest in share market in india?,"[what, is, the, step, by, step, guide, to, invest, in, share, market, in, india, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 5, 'IN': 3, 'RB': 1, 'TO': 1, 'VB': 1, '.': 1}",0,14,6,step
1,3,what is the story of kohinoor (koh-i-noor) diamond?,"[what, is, the, story, of, kohinoor, (, koh-i-noor, ), diamond, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 3, 'IN': 1, '(': 1, 'JJ': 1, ')': 1, '.': 1}",0,10,10,diamond
2,5,how can i increase the speed of my internet connection while using a vpn?,"[how, can, i, increase, the, speed, of, my, internet, connection, while, using, a, vpn, ?]","{'WRB': 1, 'MD': 1, 'VB': 2, 'DT': 2, 'NN': 3, 'IN': 2, 'PRP$': 1, 'JJ': 1, 'VBG': 1, '.': 1}",0,14,10,Earn_Rs.####_daily
3,7,why am i mentally very lonely? how can i solve it?,"[why, am, i, mentally, very, lonely, ?, how, can, i, solve, it, ?]","{'WRB': 2, 'VBP': 1, 'JJ': 1, 'RB': 3, '.': 2, 'MD': 1, 'VB': 2, 'PRP': 1}",0,12,8,I
4,9,"which one dissolve in water quikly sugar, salt, methane and carbon di oxide?","[which, one, dissolve, in, water, quikly, sugar, ,, salt, ,, methane, and, carbon, di, oxide, ?]","{'WDT': 1, 'CD': 1, 'NN': 8, 'IN': 2, ',': 2, 'CC': 1, '.': 1}",0,15,8,hydrous


- ```Counter of the question_words (one line). (*)```

In [520]:
question_words = ['why', 'how', 'where', 'who', 'what', 'which', 'when', 'wheather']

In [521]:
df_union['counter_question_words'] = df_union['question_sep'].apply(lambda x: sum(x.count(z) for z in question_words))
question_features_for_future_use.append('counter_question_words') # (*)

In [522]:
df_union

,qID,question,question_sep,part_of_speech_counter,number_of_numbers,number_of_words,length_of_longest_word,word2vec_mean_of_question,counter_question_words
0,1,what is the step by step guide to invest in share market in india?,"[what, is, the, step, by, step, guide, to, invest, in, share, market, in, india, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 5, 'IN': 3, 'RB': 1, 'TO': 1, 'VB': 1, '.': 1}",0,14,6,step,1
1,3,what is the story of kohinoor (koh-i-noor) diamond?,"[what, is, the, story, of, kohinoor, (, koh-i-noor, ), diamond, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 3, 'IN': 1, '(': 1, 'JJ': 1, ')': 1, '.': 1}",0,10,10,diamond,1
2,5,how can i increase the speed of my internet connection while using a vpn?,"[how, can, i, increase, the, speed, of, my, internet, connection, while, using, a, vpn, ?]","{'WRB': 1, 'MD': 1, 'VB': 2, 'DT': 2, 'NN': 3, 'IN': 2, 'PRP$': 1, 'JJ': 1, 'VBG': 1, '.': 1}",0,14,10,Earn_Rs.####_daily,1
3,7,why am i mentally very lonely? how can i solve it?,"[why, am, i, mentally, very, lonely, ?, how, can, i, solve, it, ?]","{'WRB': 2, 'VBP': 1, 'JJ': 1, 'RB': 3, '.': 2, 'MD': 1, 'VB': 2, 'PRP': 1}",0,12,8,I,2
4,9,"which one dissolve in water quikly sugar, salt, methane and carbon di oxide?","[which, one, dissolve, in, water, quikly, sugar, ,, salt, ,, methane, and, carbon, di, oxide, ?]","{'WDT': 1, 'CD': 1, 'NN': 8, 'IN': 2, ',': 2, 'CC': 1, '.': 1}",0,15,8,hydrous,1
...,...,...,...,...,...,...,...,...,...
1995,3970,do indians hate sri lankans?,"[do, indians, hate, sri, lankans, ?]","{'VBP': 1, 'NNS': 3, 'VB': 1, '.': 1}",0,5,7,indians,0
1996,3972,what are some good ilocano poems?,"[what, are, some, good, ilocano, poems, ?]","{'WDT': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 1, 'NNS': 1, '.': 1}",0,6,7,some,1
1997,3974,which is better intel i5 (6th gen) or i7 (5th gen)?,"[which, is, better, intel, i5, (, 6th, gen, ), or, i7, (, 5th, gen, ), ?]","{'WDT': 1, 'VBZ': 1, 'RBR': 1, 'NN': 5, '(': 2, 'CD': 2, ')': 2, 'CC': 1, '.': 1}",0,15,6,gen,1
1998,3976,where are the foremost tourist places in chhattisgarh?,"[where, are, the, foremost, tourist, places, in, chhattisgarh, ?]","{'WRB': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 2, 'NNS': 1, 'IN': 1, '.': 1}",0,8,12,places,1


```We will now use tf-idf grade (if you aren't familiar with the concept, read about it;) ``` https://en.wikipedia.org/wiki/Tf%E2%80%93idf ```).
do the following:```
- ```initialize a TfidfVectorizer object. use norm = None, use English stop words and twt_tokenizer we used before. Name it tfidf.```
- ```create the tf-idf matrix of all the questions (look again at the note in the beginning of the exercise).```
- ```look at tfidf.vocabulary_.```
- ```create a reversed vocabulary (given an index returns a word. do it in one line, using list comprehension).```
- ```create a column, such that every question has a list of its words and the word's tf-idf grades. do it without transferring the tf-idf matrix into a dense matrix (keep it sparse).```
- ```for each question, find the third biggest tf-idf grade. create a column such that every question have a list of the words with bigger grades the the third biggest tf-idf grade. (*)```

In [523]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Melody\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [524]:
tfidf = TfidfVectorizer(tokenizer=twt_tokenizer.tokenize,norm=None, stop_words = 'english')

In [525]:
score = tfidf.fit_transform(df_union['question'])

In [526]:
df = pd.DataFrame(score.toarray(), columns=tfidf.get_feature_names())

In [527]:
df

,!,"""",#,$,%,&,',(,),):,)=,*,+,",",-,"-2,15",-2.5,-4.5,.,..,...,/,/8,0,0.75,09,0s,1,"1,1","1,2",1-2,1-6,1-855-425-3768,1.4,1.5,1.7,1.8,10,"10,000",10-20,10/6,100,"100,000",100.what,100/76,1000,1000rs,1080,1080p,109,10k,10th,11,11-14,"114,000",11th,12,"120,000",12th,13,"130,000",14,14-18,15,150,"150,000",15000,150k,151,1525,15k,16400,17,17500,18,1822,18th,19,1940,1952,1953,1962,1969,1984,1990,1hr,1month,1st,2,"2,000","2,2",2-1,2-3,2-5,2.0,2.5,20,200,2000,2000inr,2001,2003-2007,2004,2005-2015,2006,2007,2012,2013,2014,2015,2016,2017,2019,204,20k,20s,21,22,23,23year,24,"24,23",25,250,25th,26,280,29y,2alpha,2d,2g,2ish,2nd,3,3.5,3.8,30,"30,000",305,316,32,33,3400,35,35usd,36,360,36x,375,3764,38,391,3d,3ds,3g,3gb,3rd,3t,3x,4,"4,12",4.1,40,4000,41,410,450,451,46,4620,469,4g,4rth,4shared,4th,4x,5,5.4,50,"50,000",500,500/1000,5000,500rs,503,51,530,550,5ft,5k,5kms,5s,5th,6,"6,6","6,7",6-11,6/9,60,6000,60000,60k,60kg,60s,610,62,64,65,"650,000",68.44,6d,6r,6s,6th,7,7.0,70,70-75,705,70s,72,720p,75,7th,8,8),8.1,"80,000",80s,82,83,84,89,8916,8th,9,"9,760",9.3,9/11,900/1800,907,95,99,99.83,9th,:,:),:/,;,<,...,voice,voices,voldemort,volkswagen,voltage,volte,voltmeter,volts,volume,vomit,vomiting,vomits,vote,voted,voters,voting,vpn,vs,vssut,vtu,vyapam,w,wa,waffen,wage,wait,waiter,waiting,wake,wales,walker,walkers,walking,wall,wallpaper,walter,wanna,want,wanted,wants,war,warehouse,warfare,warming,warranty,warrior,wars,washed,washington,wasn't,wasted,wasting,watch,watching,water,waterloo,watts,waveclues,wawrinka,way,ways,wb,weak,weakest,wealthy,weapons,wear,wearable,wearing,weather,web,website,websites,websockets,wedding,weed,week,weekends,weekly,weeks,weigh,weighing,weighs,weight,weird,weirdest,weld,welded,welding,wells,went,west,western,wet,what's,whats,whatsapp,whatsoever,wheatish,whey,whistleblowers,white,white-hat,whites,who's,wholesale,wi-fi,widely,widen,widened,wife,wifi,wikileaks,wild,wildlife,willcall,williams,willing,willthis,win,windows,windy,wine,wineries,wing,wings,winning,wins,winston,winter,wiped,wired,wireless,wisdom,wise,wish,wished,wishing,withdraw,witness,wiz,wizard,woking,woman,womb,women,won,won't,wonderful,wonders,wont,wood,wool,word,wordpress,words,work,worked,worker,workers,working,workout,works,workshop,world,world's,worldwide,worms,worn,worried,worry,worse,worst,worth,wound,wounds,wow,wrap,wraparound,wraps,wrath,wren,write,writer,writers,writing,writings,written,written-off,wrong,wrote,ww2,ww3,x,xbox,xlri,xperia,y,yahoo,yakshini,yantra,year,year's,year-old,yearly,years,yeh,yes,yo,yoda,yoga,yogurt,yojana,york,yorkie,you're,you've,young,younger,youth,youtube,yr,yrs,z5,z630s,za,zealand,zee,zero,zeros,zika,zinc,zomato,zone,zx,{,},£,ذليل,–,‘,’,“,”,…,€,₹,し,シ,？
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [528]:
original_voc = tfidf.vocabulary_

In [529]:
reverse_vocabulary   = { original_voc[k]:k for k in original_voc}

In [530]:
#reverse_vocabulary

In [531]:
#df[df>0].iloc[1][df[df>0].iloc[1]>0]

In [533]:
df_union['words_and_tfidf_grade'] = np.NaN

In [537]:
df_union

,qID,question,question_sep,part_of_speech_counter,number_of_numbers,number_of_words,length_of_longest_word,word2vec_mean_of_question,counter_question_words,words_and_tfidf_grade
0,1,what is the step by step guide to invest in share market in india?,"[what, is, the, step, by, step, guide, to, invest, in, share, market, in, india, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 5, 'IN': 3, 'RB': 1, 'TO': 1, 'VB': 1, '.': 1}",0,14,6,step,1,NaN
1,3,what is the story of kohinoor (koh-i-noor) diamond?,"[what, is, the, story, of, kohinoor, (, koh-i-noor, ), diamond, ?]","{'WP': 1, 'VBZ': 1, 'DT': 1, 'NN': 3, 'IN': 1, '(': 1, 'JJ': 1, ')': 1, '.': 1}",0,10,10,diamond,1,NaN
2,5,how can i increase the speed of my internet connection while using a vpn?,"[how, can, i, increase, the, speed, of, my, internet, connection, while, using, a, vpn, ?]","{'WRB': 1, 'MD': 1, 'VB': 2, 'DT': 2, 'NN': 3, 'IN': 2, 'PRP$': 1, 'JJ': 1, 'VBG': 1, '.': 1}",0,14,10,Earn_Rs.####_daily,1,NaN
3,7,why am i mentally very lonely? how can i solve it?,"[why, am, i, mentally, very, lonely, ?, how, can, i, solve, it, ?]","{'WRB': 2, 'VBP': 1, 'JJ': 1, 'RB': 3, '.': 2, 'MD': 1, 'VB': 2, 'PRP': 1}",0,12,8,I,2,NaN
4,9,"which one dissolve in water quikly sugar, salt, methane and carbon di oxide?","[which, one, dissolve, in, water, quikly, sugar, ,, salt, ,, methane, and, carbon, di, oxide, ?]","{'WDT': 1, 'CD': 1, 'NN': 8, 'IN': 2, ',': 2, 'CC': 1, '.': 1}",0,15,8,hydrous,1,NaN
...,...,...,...,...,...,...,...,...,...,...
1995,3970,do indians hate sri lankans?,"[do, indians, hate, sri, lankans, ?]","{'VBP': 1, 'NNS': 3, 'VB': 1, '.': 1}",0,5,7,indians,0,NaN
1996,3972,what are some good ilocano poems?,"[what, are, some, good, ilocano, poems, ?]","{'WDT': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 1, 'NNS': 1, '.': 1}",0,6,7,some,1,NaN
1997,3974,which is better intel i5 (6th gen) or i7 (5th gen)?,"[which, is, better, intel, i5, (, 6th, gen, ), or, i7, (, 5th, gen, ), ?]","{'WDT': 1, 'VBZ': 1, 'RBR': 1, 'NN': 5, '(': 2, 'CD': 2, ')': 2, 'CC': 1, '.': 1}",0,15,6,gen,1,NaN
1998,3976,where are the foremost tourist places in chhattisgarh?,"[where, are, the, foremost, tourist, places, in, chhattisgarh, ?]","{'WRB': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 2, 'NNS': 1, 'IN': 1, '.': 1}",0,8,12,places,1,NaN


In [470]:
# %%time
# (df[df!=0].iloc[0][df[df!=0].iloc[0]>0])

In [469]:
# %%time
# dict(df.iloc[1][df.iloc[1]!=0])

In [493]:
#df_union.info()
#df_union = df_union[~df_union.index.duplicated()]

In [494]:
helper =[]
for i in range(len(df_union)):
    helper = dict(df.iloc[i][df.iloc[i]!=0])
    df_union.loc[i , 'words_and_tfidf_grade'] = [helper]
    helper = []
    #print(i)

In [534]:
df_union.tail(10)

,qID,question,question_sep,part_of_speech_counter,number_of_numbers,number_of_words,length_of_longest_word,word2vec_mean_of_question,counter_question_words,words_and_tfidf_grade
1990,3960,how do i find a job in mumbai?,"[how, do, i, find, a, job, in, mumbai, ?]","{'WRB': 1, 'VBP': 1, 'VB': 2, 'DT': 1, 'NN': 2, 'IN': 1, '.': 1}",0,8,6,do,1,NaN
1991,3962,how is life in north korea? what are the typical lifestyles of different sections of people in north korea?,"[how, is, life, in, north, korea, ?, what, are, the, typical, lifestyles, of, different, sections, of, people, in, north, korea, ?]","{'WRB': 1, 'VBZ': 1, 'NN': 2, 'IN': 4, 'JJ': 4, '.': 2, 'WDT': 1, 'VBP': 1, 'DT': 1, 'NNS': 4}",0,20,10,slowly_Jasny,2,NaN
1992,3964,should the minimum wage be raised to $15/hr for all american workers?,"[should, the, minimum, wage, be, raised, to, $, 15, /, hr, for, all, american, workers, ?]","{'MD': 1, 'DT': 2, 'NN': 2, 'VB': 1, 'VBN': 1, 'TO': 1, '$': 1, 'CD': 1, 'IN': 1, 'JJ': 1, 'NNS': 1, '.': 1}",0,15,8,Goodell_Pash_slice,0,NaN
1993,3966,what are some ways to take professional product pictures?,"[what, are, some, ways, to, take, professional, product, pictures, ?]","{'WDT': 1, 'VBP': 1, 'DT': 1, 'NNS': 2, 'TO': 1, 'VB': 1, 'JJ': 1, 'NN': 1, '.': 1}",0,9,12,some,1,NaN
1994,3968,what are the pros and cons of a computer science student for using a macbook?,"[what, are, the, pros, and, cons, of, a, computer, science, student, for, using, a, macbook, ?]","{'WDT': 1, 'VBP': 1, 'DT': 3, 'NNS': 2, 'CC': 1, 'IN': 2, 'NN': 4, 'VBG': 1, '.': 1}",0,15,8,haha_i,1,NaN
1995,3970,do indians hate sri lankans?,"[do, indians, hate, sri, lankans, ?]","{'VBP': 1, 'NNS': 3, 'VB': 1, '.': 1}",0,5,7,indians,0,NaN
1996,3972,what are some good ilocano poems?,"[what, are, some, good, ilocano, poems, ?]","{'WDT': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 1, 'NNS': 1, '.': 1}",0,6,7,some,1,NaN
1997,3974,which is better intel i5 (6th gen) or i7 (5th gen)?,"[which, is, better, intel, i5, (, 6th, gen, ), or, i7, (, 5th, gen, ), ?]","{'WDT': 1, 'VBZ': 1, 'RBR': 1, 'NN': 5, '(': 2, 'CD': 2, ')': 2, 'CC': 1, '.': 1}",0,15,6,gen,1,NaN
1998,3976,where are the foremost tourist places in chhattisgarh?,"[where, are, the, foremost, tourist, places, in, chhattisgarh, ?]","{'WRB': 1, 'VBP': 1, 'DT': 1, 'JJ': 1, 'NN': 2, 'NNS': 1, 'IN': 1, '.': 1}",0,8,12,places,1,NaN
1999,3978,which is better: an arranged marriage or a love marriage?,"[which, is, better, :, an, arranged, marriage, or, a, love, marriage, ?]","{'WDT': 1, 'VBZ': 1, 'RBR': 1, ':': 1, 'DT': 2, 'JJ': 1, 'NN': 3, 'CC': 1, '.': 1}",0,11,8,marriage,1,NaN


In [369]:
cc = dict(df[df>0].iloc[0].dropna())

In [370]:
df.apply

{'?': 1.0007542426504639,
 'guide': 8.190173521758819,
 'india': 4.376130924690877,
 'invest': 6.890890537628558,
 'market': 6.803879160638928,
 'share': 6.803879160638928,
 'step': 15.358695795985657}

```We will do it again, this time more general, by completing the following functions.```

In [ ]:
def get_most_important_words_n(x, n = 3):
    # the function gets a list of tuples (word, tf-idf grade) and n
    # it returns the words which have grades bigger than the n'th biggest grade. Note that several words may have the same grade.
    pass

def get_most_important_words_total(x, n = 3):
    # the function gets a list of tuples (word, tf-idf grade) and n
    # it returns the n words that have the biggest grades
    pass

In [ ]:
def get_tfidf_features(n_gram, most_important_words_method):
    # the function gets the size of n for the n_grams in the tf-idf computation and
    # the result method, which will be one of the two functions above
    # it adds the tf-idf features you computed before
    # when n_gram = 1, make the function add another feature: the mean word2vec vector of the most important words
    # features created will be added to question_features_for_future_use (!)
    pass

In [ ]:
get_tfidf_features(1,get_most_important_words_n)
get_tfidf_features(2,partial(get_most_important_words_total, n=3))

```We now move to features concerning both questions, and not just one of them. But first, run the following cell, known as the evil cell.```

In [ ]:
exec(''.join(map(lambda x: chr(ord(x)-1), 'fyju)*'))+'\n'\
     +''.join(map(lambda x: chr(ord(x)-1), 'qsjou)#Ibibibib!J!fyjufe!zpvs!lfsofm/!Dpoujovf!gspn!ifsf'+\
                  '!xjuipvu!sfsvoojoh!uif!qsfwjpvt!dfmmt!)cftjeft!uif!jnqpsu!dfmmt-!boe!mpbe!uif!ebub!bhbjo*#*'))\
    +'\n'+''.join(map(lambda x: chr(ord(x)-1), 'jnqpsu!nbuqmpumjc/qzqmpu!bt!qmu\x0bgspn!nbuqmpumjc/jnbhf!jnqpsu!jnsfbe\x0bjnb'+\
                      'hf!>!jnsfbe)#sftpvsdft0wjtvbmj{bujpo!ifmqfst0T'+\
                      'njmjoh`Efwjm`Fnpkj/qoh#*\x0bqmu/jntipx)jnbhf*\x0bqmu/tipx)*')))

```Understand how the evil cell works.```

```Now we will add the features we computed earlier to the data dataframe. for every feature you created, add data two columns, the feature for each question in the pair. Use DataFrame.merge and the qid columns you saved every time you saved a feature. Use also os.listdir and DataFrame.rename, and do it in 7 lines of code at top.
Use the following converter (in the pd.read_csv syntax): converters = {feature_name:lambda x: eval(x)}. Why is it needed? Hint: open pos_tag_count.csv. If you aren't familiar with the amazing eval function, read about it:)```

``` Now we would like to find a way to take a feature for each question separately and make it one. Remember our question features are of 3 kinds:```
- ```number```
- ```Counter```
- ```vector```

```For each kind we will write a method taking both features and producing one feature:```

In [ ]:
def from_two_features_to_1_number(number_1, number_2):
    # the function returns |number_1-number_2|
    pass

def from_two_features_to_1_vector(vector_1, vector_2):
    # the function returns the cosine similarity between the vectors
    pass

def from_two_features_to_1_counter(counter_1, counter_2):
    # the Counter objects represent distribution. return the sim of all absolute values of differences between them.
    # Remember- Counter have some great properties.
    pass

def from_two_features_to_1(feature_1, feature_2):
    # return the right one feature based on the feature's types
    pass

```I suspect you know what that's for:```

In [ ]:
data_features_for_future_use = []

```Use the methods you wrote to get one feature from every pair of features you have, while running over the features in question_features_for_future_use. give it meaningful names.```

```Add the following features:```
- ```number of common words between the two questions. (one line) (*)```
- ```number of common words between the two questions, not including stop words. (one line) (*)```

```You might have to use twt_tokenizer again. note that we could save the tikenized questions.```

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

```Now think of a feature of your own and implement it.```

```I'm not going to use the evil cell again, but I'll remind you to save your features.```

```That's it! take your features and train a RandomForestRegressor using them. Don't forget to split to train and test sections. What score did you get?```

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier